In [0]:
from pyspark.sql.functions import col, sum, avg, round, bround

In [0]:
# Load a Golden

dbutils.widgets.text("catalog", "dev_ecommerce")
dbutils.widgets.text("storage_account","adlsproyecto2025")

In [0]:
catalog_param = dbutils.widgets.get("catalog")
storage_account = dbutils.widgets.get("storage_account")
# container y schema serán lo mismo
container = {"origen":"silver","destino":"golden"}

In [0]:
df_silver_procesar = spark.table(f"{catalog_param}.{container['origen']}.sales_orders")

In [0]:
# Calcular métricas de negocio agregadas por categoría y segmento de cliente

# crear columan venta bruta
df_silver_procesar_summary = df_silver_procesar.withColumn(
    "gross_sale",bround((col("Price")*col("Units_Sold")),2)
)
# crear columan sales_total

df_silver_procesar_summary = df_silver_procesar_summary.withColumn(
    "total_sales",bround((col("Price")*col("Units_Sold"))-col("Discount"),2)
)

In [0]:
df_silver_procesar_summary.show(5)

In [0]:
# ruta destino
golden_path = f"abfss://{container['destino']}@{storage_account}.dfs.core.windows.net/sales_orders/"

In [0]:
# creacion de los archivos delta

df_silver_procesar_summary.write.format("delta").mode("overwrite").save(golden_path)
                                                                             

In [0]:
# creacion de la tabla sales_ordes en el schema golden

spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {catalog_param}.{container['destino']}.sales_orders
    USING DELTA
    LOCATION '{golden_path}'
""")